In [1]:
import os 
os.environ['KERAS_BACKEND']='tensorflow'

# ## Use tensorflow with CPU
# import tensorflow as tf
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
# sess = tf.Session(config=config)
# from keras import backend as K
# K.tensorflow_backend.set_session(session=sess)
# ##

In [2]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

Using TensorFlow backend.


Import time: 6.081101179122925
Import time: 0.0001709461212158203


In [3]:
global_start = time()

In [4]:
train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_folds, valid_folds = stratified_kfold_sampling(Y, n_splits=10, random_state=1000)
print("Shape of train folds {}".format(train_folds.shape))
print("Shape of valid folds {}".format(valid_folds.shape))

image_size = 128

Shape of Y (40479, 17)
Shape of train folds (10,)
Shape of valid folds (10,)


In [5]:
model_name = "deep_model_vgg19"

fold_inx = 1
batch_size = 32
all_steps = np.ceil(len(Y)/batch_size)
train_steps = np.ceil(len(train_folds[fold_inx])/batch_size)
valid_steps = np.ceil(len(valid_folds[fold_inx])/batch_size)
test_steps = np.ceil(len(test_df)/batch_size)

In [6]:
x_input = Input(shape=(image_size, image_size, 3))
x = BatchNormalization()(x_input)

vgg19_model = get_pretrained_model('vgg19', (image_size, image_size, 3))
vgg19_model.layers.pop() # removing last MaxPooling layer
vgg19_model = Sequential(vgg19_model.layers) # reinitialize

x = vgg19_model(x)

# TODO?

xa = GlobalAveragePooling2D()(x)
xm = GlobalMaxPooling2D()(x)
x = concatenate([xm, xa])

x = Dense(512, activation='relu')(x)

x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)

x_output = Dense(17, activation='sigmoid')(x)

In [7]:
model = Model(inputs=x_input, outputs=x_output)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 128, 128, 3)   12          input_1[0][0]                    
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 8, 8, 512)     20024384    batch_normalization_1[0][0]      
____________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalMa (None, 512)           0           sequential_1[1][0]               
___________________________________________________________________________________________

In [8]:
def getImageDataGenerator():
    return ImageDataGenerator(
                rotation_range=90,
                horizontal_flip=True,
                vertical_flip=True,)

all_gen = getImageDataGenerator()
train_gen = getImageDataGenerator()
valid_gen = getImageDataGenerator()
test_gen = getImageDataGenerator()

In [9]:
# del model
model = load_model('../output/vgg19/deep_model_vgg19_1_LB092919.h5', compile=False)

In [10]:
import os
import sys

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)
    
os.environ['THEANO_FLAGS'] = 'device=cpu'

from data_utils import TRAIN_CSV_FILEPATH, TRAIN_CSV, TRAIN_ENC_CSV, get_caption, train_jpg_ids, unique_tags
from image_utils import get_image_data    

In [11]:
from glob import glob
from data_utils import GENERATED_DATA

gen_train_jpg_files = glob(os.path.join(GENERATED_DATA, "train", "jpg", "*.jpg"))
gen_train_jpg_ids = [s[len(os.path.join(GENERATED_DATA, "train", "jpg"))+1+len('gen_train_'):-4] for s in gen_train_jpg_files]

In [12]:
id_type_list = [(image_id, "Generated_Train_jpg") for image_id in gen_train_jpg_ids]

In [13]:
from data_utils import get_label

In [14]:
y_true = np.zeros((len(id_type_list), len(unique_tags)), dtype=np.uint8)
counter = 0
for image_id, image_type in id_type_list:
    y_true[counter,:] = get_label(image_id, image_type)
    counter += 1

In [15]:
y_true[:10,:]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]], dtype=uint8)

In [21]:
test_array = np.zeros((len(id_type_list), image_size, image_size, 3), dtype=np.float32)

In [22]:
import cv2

counter = 0
for image_id, image_type in id_type_list:
    img = get_image_data(image_id, image_type)
    img = cv2.resize(img, dsize=(128, 128))
    img = img.astype(np.float32) / 255.0
    test_array[counter,:, :, :] = img
    counter += 1

In [23]:
thres = load_array('../output/vgg19/deep_model_vgg19_thres_1.dat')

In [25]:
test_array.shape, test_array.dtype, test_array.max()

((8500, 128, 128, 3), dtype('float32'), 1.0)

In [26]:
p_X_test_no_tta, p_X_test_tta_list = predictTTA(model, test_array, test_gen, test_steps, batch_size, 3)
p_X_test_tta = sum(p_X_test_tta_list)

8500/8500 [==============================] - 536s   
Predicting TTA: 0
  54/1913 [..............................] - ETA: 4218s

KeyboardInterrupt: 

In [57]:
c = 3.0
p_X_test = (c*p_X_test_no_tta+p_X_test_tta)/(c+3.0)
p_X_test = p_X_test_no_tta

In [58]:
start = time()
for inx in np.arange(len(test_df)):
    test_df['tags'][inx] = ' '.join([inv_label_map[i] for i, b in enumerate(p_X_test[inx] >= thres) if b])
print(time() - start)

test_df[:10]

14.894867181777954


,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,agriculture clear cultivation primary
4,test_4,cloudy partly_cloudy primary
5,test_5,clear primary
6,test_6,agriculture clear cultivation habitation partl...
7,test_7,agriculture clear habitation primary road
8,test_8,clear primary
9,test_9,agriculture clear cultivation haze primary


In [59]:
save_array(DATA_DIR+'/results/'+model_name+'_test.dat', p_X_test)
test_df.to_csv(DATA_DIR+'/results/'+model_name+'_submission.csv', index=False)
FileLink('data/results/'+model_name+'_submission.csv')

/src/DL/planet_understanding_the_amazon_from_space/data/results/deep_model_vgg19_submission.csv

In [41]:
print((time() - global_start)/60)

69.27662885983786
